In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.transform import rescale
from scipy.signal import convolve2d
import socket 
import sys
import time
from skimage import io

def rgb_convolve2d(image, kernel):
    red = convolve2d(image[:,:,0], kernel, 'valid')
    green = convolve2d(image[:,:,1], kernel, 'valid')
    blue = convolve2d(image[:,:,2], kernel, 'valid')
    return np.stack([red, green, blue], axis=2)

def my_conv(image_name):
    my_dog = imread(image_name)
    r_scaled = rescale(my_dog[:,:,0], 0.10)
    g_scaled = rescale(my_dog[:,:,1], 0.10)
    b_scaled = rescale(my_dog[:,:,2], 0.10)
    my_dog_scaled = np.stack([r_scaled, g_scaled, b_scaled], axis=2)
    my_dog_gray = rescale(rgb2gray(my_dog), 0.10)
    ##
    identity = np.array([[0, 0, 0],
                         [0, 1, 0],
                         [0, 0, 0]])
    conv_im1 = rgb_convolve2d(my_dog_scaled, identity)
    fig, ax = plt.subplots(1,2, figsize=(12,5))
    ax[0].imshow(identity, cmap='gray')
    ax[1].imshow(abs(conv_im1), cmap='gray');
    
    ##
    with open("result.jpeg", "wb") as g :
        # write down the stream of bytes as image
        g.write(conv_im1)

    

In [ ]:
if __name__ == "__main__":
    HOST = "localhost"
    PORT = 5005
    try:
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print("\n[*] Socket successfully created")
    except socket.error as err:
        print("\n[*] Socket creation failed with error : ",err)
    try:
        server_socket.bind((HOST, PORT))
    except socket.error as msg:
        print('Bind failed. Error  : {} '.format(msg))
        sys.exit()
    server_socket.listen(1)
    print("\n[*] Socket is now listening")
    # each iteration read/send 4 k-bytes
    CHUNK_SIZE = 4 * 1024  
    while True:
        Client_Socket, addr = server_socket.accept()
        print('\n[*] Connected from ip: {} and port : {} '.format(addr[0],addr[1]))
        SND_Bytes= b""
        res = b''
        # receive the image as a stream of bytes
        # if the server receives b'0' ==> end of image
        while not (SND_Bytes == b'0'):
            SND_Bytes = Client_Socket.recv(CHUNK_SIZE)
            res += SND_Bytes
        # w: write -- b: binary
        with open("image_from_client.jpg", "wb") as g :
            # write down the stream of bytes as image
            g.write(res)
        # conv work
        conv_image=my_conv('image_from_client.jpg')
        
        
        #print(grayscale.shape)
        
        
        #print(imgconvol)

        print("Image received and recreated")
        # r: read -- b: binary
        f = open("result.jpg", "rb")
        # print(len(f))
        print("Sending image data...")
        # sleep untill the client invoke the bloking recv call
        time.sleep(0.5)
        NB_Bytes = 1
        
        while NB_Bytes>0:
            data = f.read(CHUNK_SIZE)
            NB_Bytes = Client_Socket.send(data)
            print("Length Data sent:"+str(NB_Bytes))    
            
        # send the bytes "0" to indicate the end of the image
        Client_Socket.send(b'0')
        Client_Socket.close()
        print("\n----------------Connection Closed-------------------\n")


[*] Socket successfully created

[*] Socket is now listening

[*] Connected from ip: 127.0.0.1 and port : 40750 
Image received and recreated
Sending image data...
Length Data sent:4096
Length Data sent:4096
Length Data sent:1408
Length Data sent:0

----------------Connection Closed-------------------

